In [1]:
import pandas as pd
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
import dash
from dash import Dash, html, dcc
# import dash_core_components as dcc 
# import dash_html_components as html
from dash.dependencies import Input, Output, State
import datetime

# from visualize import Visualize2
import random
# from google.cloud import bigquery
# from google.oauth2 import service_account
import pandas as pd
import numpy as np
import json

In [2]:
from urllib.request import urlopen

In [3]:
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
'./chain-coders-5d47d58b2c33.json')

project_id = 'chain-coders'
client = bigquery.Client(credentials= credentials,project=project_id)
# query = "SELECT * FROM `redfin.redfin-data-state`"
# df = client.query(query).to_dataframe()

In [4]:
app = dash.Dash(__name__)

In [5]:
# -- Import and clean data (importing csv into pandas)
# HPI_state = pd.read_csv('./data/HPI_AT_state.csv', header=None)
# HPI_state.columns = ['state_abbr', 'year', 'quarter', 'HPI_base_1980']
# HPI_state['year_quarter'] = HPI_state.year.astype(str)[:]+'-Q'+HPI_state.quarter.astype(str)[:]
# HPI_state.year_quarter = HPI_state.year_quarter.str.replace(r'(\d{4})_(\d)', r'\1-Q\2')
# HPI_state.year_quarter = pd.to_datetime(HPI_state.year_quarter).dt.date
# df = HPI_state.reindex(['state_abbr', 'year_quarter', 'HPI_base_1980'], axis=1)
query = "SELECT * FROM `redfin.HPI_county` "
HPI_county = client.query(query).to_dataframe()
HPI_county.dropna(subset=['HPI_base_2000'], inplace=True)
HPI_county.year = pd.to_datetime(HPI_county[['year']].assign(day=1, month=1)).dt.date
HPI_county.sort_values(['fips_code', 'year'], inplace=True)
HPI_county.reset_index(drop=True, inplace=True)

In [6]:
df = HPI_county

In [7]:
#property types for drop down
property_set = ['']

#metric types for dropdown
metric_set = ['HPI_base_2000']

#dates for slider
sorted_dates = np.sort(df['year'].unique())
sorted_dates = ['{}'.format(i) for i in sorted_dates] #convert each date to string within a list
counter = 0
year = 1975

slider_labels = {}
#labels for slider
for index, date in enumerate(sorted_dates):
       if counter == index:
              slider_labels['{}'.format(index)] = year
              year += 1
              counter += 1
       else: 
              slider_labels['{}'.format(index)] = ''

# labels = {index: date for index, date in enumerate(sorted_dates)} #labels of dates for slider object

#color-scale
color_scale = [[0.0, '#edf8fb'],[0.2, '#ccece6'],[0.4, '#99d8c9'],
       [0.6, '#66c2a4'],[0.8, '#2ca25f'],[1.0, '#006d2c']] # purples

In [ ]:
print(property_set)

In [ ]:
print(sorted_dates)
print(len(sorted_dates))

In [ ]:
# line = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2])])

In [8]:
df2 = pd.read_csv('pred_HPI_county.csv')

In [9]:
df2.year = pd.to_datetime(df2.year)

In [10]:
df2.fips_code = df2.fips_code.astype(str).str.zfill(5)

In [ ]:
# temp = df2[(df2['property'] == 'All Residential') & (df2['state'] == 'MN')]
# print(temp)

In [11]:
# ------------------------------------------------------------------------------
# App layout
#what goes inside dash layout are the dash components and any html we need
app.layout = html.Div([

    html.H1("HPI County Choropleth Data", style={'text-align': 'center'}), #html Title

#     html.Div(id='output_container', children=["Metric"], style = {'font-weight': 'bold'}),

    #components, e.g., drop downs, sliders
    #first drop down by metric
    dcc.Dropdown(id="metric",
                 placeholder = "Select Metric",
                 options=metric_set,
                 value=metric_set[0],
                 style={'width': "60%",}
                 ),

    html.Br(), #space between dropdowns

#     html.Div(id='output_container', style = {'font-weight': 'bold'}, 
#     children = ["Property Type"]
#     ),

    #second drop down for property type
    dcc.Dropdown(id="property_type",
                 placeholder = "Select Property Type",
                 options=property_set,
                 value=property_set[0],
                 style={'width': "60%"},
                 ),

    
    html.Br(), #space
    
    #create div element, e.g., text to display dropdown selection
    html.Div(id='output_container', children=[]),

    html.Br(), #space

    #graph object, e.g., choropleth
    dcc.Graph(id='choropleth', figure={}),

    

    #slider object
    dcc.Slider(0, len(sorted_dates), step = None, #min value of 0 and max of number of unique dates
               value=0, #where the slider starts
               marks = slider_labels,
               tooltip={"placement": "bottom", "always_visible": True}, #place slider at bottom of graph
               id='date_chosen'),

    #add line chart
    dcc.Graph(id = "line_chart", figure = {})

])

In [12]:
#get geojson for counties
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_geojson = json.load(response)

In [13]:
@app.callback(
    Output(component_id='output_container', component_property='children'),
    Output(component_id='choropleth', component_property='figure'),
    Output(component_id = 'line_chart', component_property = 'figure'),
    Input(component_id='property_type', component_property='value'),
    Input(component_id = 'metric', component_property = 'value'),
    Input(component_id='date_chosen', component_property='value'),
    Input(component_id = 'choropleth', component_property = 'hoverData'))


def update_graph(property_type, metric, date_chosen, hover):
    print(property_type)
    print(type(property_type))
    # print(hover)
    # print(type(hover))

    container = "Date Selected: {}".format(sorted_dates[date_chosen])

    dff = df.copy()
    dff = dff[dff['year'] == pd.to_datetime(sorted_dates[date_chosen])]

    # Plotly Express
    fig = px.choropleth(
        data_frame = dff,
        geojson = counties_geojson,
        featureidkey="id",
        locations='fips_code',
        scope="usa",
        color='{}'.format(metric),
        hover_data=['fips_code', '{}'.format(metric)],
        color_continuous_scale= color_scale
    )

    if type(hover) == dict:
        hover_fips = hover['points'][0]['customdata'][0]
        hover_df = df2[df2['fips_code'] == hover_fips]
        # hover_df = hover_df[['year', '{}'.format(metric)]]
        # hover_x = hover_df['year'].values.tolist()
        # hover_y = hover_df['{}'.format(metric)].values.tolist()
        # hover_y = [int(year) for year in hover_y]
        line_data = px.line(hover_df, x = "year", y = ['predicted', 'actual'], title = '{fmetric} for {fstate}'.format(fmetric = metric, fstate = hover_fips))
        # line_data = go.Figure(data=[go.Scatter(x=hover_x, y=hover_y)])
        result = line_data
    else:
        result = []

    return container, fig, result

In [16]:
df2

,year,predicted,fips_code,actual
0,2020-01-01,137.717766,01001,136.60
1,2021-01-01,139.066462,01001,148.07
2,2020-01-01,177.435069,01003,173.47
3,2021-01-01,180.547513,01003,195.93
4,2020-01-01,128.831624,01005,124.75
...,...,...,...,...
4815,2021-01-01,183.903944,56035,195.49
4816,2020-01-01,194.263321,56043,186.50
4817,2021-01-01,194.882518,56043,201.59
4818,2020-01-01,217.882076,56045,211.64


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [04/Dec/2022 11:52:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 11:52:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 11:52:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 11:52:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Dec/2022 11:52:31] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Dec/2022 11:52:31] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Dec/2022 11:52:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Dec/2022 11:52:31] "/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will 


<class 'str'>


127.0.0.1 - - [04/Dec/2022 11:52:32] "POST /_dash-update-component HTTP/1.1" 200 -



<class 'str'>


...

/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

127.0.0.1 - - [04/Dec/2022 11:53:30] "POST /_dash-update-component HTTP/1.1" 200 -



<class 'str'>


/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

127.0.0.1 - - [04/Dec/2022 11:53:37] "POST /_dash-update-component HTTP/1.1" 200 -



<class 'str'>


/Users/enrico.eduardo/opt/miniconda3/envs/plutho/lib/python3.10/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

127.0.0.1 - - [04/Dec/2022 11:53:41] "POST /_dash-update-component HTTP/1.1" 200 -
